In [1]:
#Mecab install
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 14.9 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-24 12:23:33--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::6

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#data load
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7f2a40a95eb0>)

In [4]:
#add col name
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [5]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [6]:
# 평점이 4, 5인 리뷰에는 레이블 1을, 평점이 1, 2인 리뷰에는 레이블 0을 부여합니다. 부여된 레이블은 새로 생성한 label이라는 열에 저장합니다.
total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)
total_data[:5]

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [7]:
total_data['ratings'].nunique(), total_data['reviews'].nunique(), total_data['label'].nunique()

(4, 199908, 2)

In [8]:
total_data.drop_duplicates(subset=['reviews'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거
print('총 샘플의 수 :',len(total_data))

#null 값 있는지 확인
print(total_data.isnull().values.any())

총 샘플의 수 : 199908
False


In [9]:
train, test = train_test_split(total_data, test_size = 0.25, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train))
print('테스트용 리뷰의 개수 :', len(test))

훈련용 리뷰의 개수 : 149931
테스트용 리뷰의 개수 : 49977


In [10]:
train['label'].value_counts()

1    75013
0    74918
Name: label, dtype: int64

In [11]:
print(train.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  74918
1      1  75013


In [12]:
test.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
test['reviews'] = test['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_ = test.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test))

전처리 후 테스트용 샘플의 개수 : 49977


<ipython-input-12-4d892083f662>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['reviews'] = test['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행


In [13]:
mecab = Mecab()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

train['tokenized'] = train['reviews'].apply(mecab.morphs)
train['tokenized'] = train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

test['tokenized'] = test['reviews'].apply(mecab.morphs)
test['tokenized'] = test['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [14]:
#긍정/부정 단어 빈도수 계산
negative_words = np.hstack(train[train.label == 0]['tokenized'].values)
positive_words = np.hstack(train[train.label == 1]['tokenized'].values)

In [15]:
negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

[('.', 77185), ('네요', 31941), ('는데', 20309), ('안', 19759), ('어요', 15232), ('있', 13203), ('너무', 12985), ('했', 11897), ('좋', 9813), ('배송', 9682), ('..', 9660), ('같', 9005), ('구매', 8886), ('거', 8862), ('어', 8827), ('아요', 8680), ('없', 8674), ('습니다', 8443), ('그냥', 8355), ('되', 8349)]


In [16]:
positive_word_count = Counter(positive_words)
print(positive_word_count.most_common(20))

[('좋', 39523), ('.', 35659), ('아요', 21276), ('네요', 19912), ('어요', 19351), ('잘', 18607), ('구매', 16174), ('습니다', 13340), ('있', 12391), ('배송', 12274), ('!', 12007), ('는데', 11712), ('했', 10172), ('~', 9980), ('합니다', 9826), ('먹', 9638), ('재', 9267), ('너무', 8387), ('같', 7870), ('만족', 7251)]


In [17]:
X_train = train['tokenized'].values
y_train = train['label'].values
X_test= test['tokenized'].values
y_test = test['label'].values

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 42241
등장 빈도가 1번 이하인 희귀 단어의 수: 19649
단어 집합에서 희귀 단어의 비율: 46.51641769844464
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.7794957972525441


In [19]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 22594


In [20]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [21]:
print(X_train[:3])

[[72, 146, 2082, 314, 14758, 276, 78, 7, 253, 181, 149, 837, 3061, 659, 3, 85, 68, 219, 45, 1402, 168, 4, 7], [511, 2823, 2, 2, 8837, 2663, 2, 2, 2600, 354, 3025, 267, 2429, 41, 496, 3], [50, 27, 876, 104, 39, 2430, 171, 8, 11, 8348, 5, 2, 1375, 33, 151, 336, 44, 64, 171, 151, 8, 1985, 3, 2, 4622, 121, 176, 1443, 299, 46, 32, 130, 148, 2, 4622]]


In [22]:
print(X_test[:3])

[[15, 741, 799, 126, 198, 266, 14], [354, 4057, 68, 4161, 1738], [13, 77, 3, 54, 177, 4, 30, 16, 7, 537, 304, 18, 100, 120, 587, 64, 8, 3]]


In [23]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 87
리뷰의 평균 길이 : 16.81261380234908


In [24]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [25]:
max_len = 80
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 80 이하인 샘플의 비율: 99.99866605305107


In [26]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

###GRU로 네이버 쇼핑 리뷰 감성 분류하기

In [27]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
1875/1875 [==============================] - ETA: 0s - loss: 0.2863 - acc: 0.8909
Epoch 1: val_acc improved from -inf to 0.90362, saving model to best_model.h5
1875/1875 [==============================] - 392s 207ms/step - loss: 0.2863 - acc: 0.8909 - val_loss: 0.2603 - val_acc: 0.9036
Epoch 2/15
1875/1875 [==============================] - ETA: 0s - loss: 0.2247 - acc: 0.9195
Epoch 2: val_acc improved from 0.90362 to 0.92093, saving model to best_model.h5
1875/1875 [==============================] - 407s 217ms/step - loss: 0.2247 - acc: 0.9195 - val_loss: 0.2183 - val_acc: 0.9209
Epoch 3/15
1875/1875 [==============================] - ETA: 0s - loss: 0.2031 - acc: 0.9282
Epoch 3: val_acc improved from 0.92093 to 0.92267, saving model to best_model.h5
1875/1875 [==============================] - 422s 225ms/step - loss: 0.2031 - acc: 0.9282 - val_loss: 0.2160 - val_acc: 0.9227
Epoch 4/15
1875/1875 [==============================] - ETA: 0s - loss: 0.1892 - acc: 0.9337
Epoch 4

In [28]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1562/1562 [==============================] - 45s 28ms/step - loss: 0.2189 - acc: 0.9209

 테스트 정확도: 0.9209


###리뷰 예측해보기

In [29]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [32]:
sentiment_predict('상품이 너무 좋아요.')

1/1 [==============================] - 0s 29ms/step
86.67% 확률로 긍정 리뷰입니다.


In [33]:
sentiment_predict('누가 이렇게 만드냐?ㅋㅋ')

1/1 [==============================] - 0s 44ms/step
81.77% 확률로 부정 리뷰입니다.
